In [1]:
from langdetect import detect
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
import os

In [2]:
df = pd.read_csv("../data/qald-8-test.csv")

In [3]:
df= df.dropna(subset=['questions'])

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(df['questions'])

from sklearn.feature_extraction.text import TfidfTransformer
 
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
X = tfidf_transformer.fit_transform(X_train)

# get feature names
feature_names=vectorizer.get_feature_names()

#Function for sorting tf_idf in descending order
from scipy.sparse import coo_matrix
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results
def ExtractKeyWords(text):
    #generate tf-idf for the given text
    tf_idf_vector=tfidf_transformer.transform(text)
    
    
    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())
    #extract only the top n; n here is 3
    keywords=extract_topn_from_vector(feature_names,sorted_items,3)
 
    # now print the results
    sentence = ""
    for k in keywords:
        sentence+=(" "+k)
    return sentence

new_set = []
for text in X:
    new_set.append(ExtractKeyWords(text))
    
df["questions"]=new_set

In [5]:
df_en = df[df["lang"]=='en']
df_fr = df[df["lang"]=='fr']
df_de = df[df["lang"]=='de']

In [6]:
def extractLan(f,n):
    result=[]
    count=0
    while count < n:
        line = f.readline()
        if line == "\n":
            continue
        result.append(line[:3])
        count+=1
    return result


def run_opennlp(data):
    with open('opennlp_input.txt', 'w') as f:
        for q in data["questions"]:
            f.write(q+"\n\n")
    os.system("/home/oshara/GSoC/DBPedia/openNLP/apache-opennlp-1.9.2/bin/opennlp LanguageDetector /home/oshara/GSoC/DBPedia/openNLP/langdetect-183.bin </home/oshara/GSoC/DBPedia/QLang/evaluation/opennlp_input.txt> /home/oshara/GSoC/DBPedia/QLang/evaluation/opennlp_output.txt")
    n=len(data)
    with open('opennlp_output.txt', 'r') as f:
        return extractLan(f,n)
        

In [7]:
re_en = run_opennlp(df_en)
re_fr = run_opennlp(df_fr)
re_de = run_opennlp(df_de)

In [8]:
# English, Spanish, German, Italian, French, Dutch, Romanian
# {'en': 0, 'de': 1, 'es': 2, 'it': 3, 'fr': 4, 'nl': 5, 'ro': 6}
# label = {'eng': 0, "spa": 2, "deu": 1, "ita": 3, "fra": 4, "nld": 5, "ron": 6, "other": 7}
label = {'eng': 0, "deu": 1, "fra": 2,"other": 3}

In [9]:
def replace_label(result):
    for i in range(len(result)):
        key = result[i]
        if key in label:
            result[i] = label[key]
        else:
            result[i] = 7

replace_label(re_en)
replace_label(re_de)
replace_label(re_fr)

In [10]:
possible_labels = df["lang"].unique()

label_dict = {"en":0,"de":1,"fr":2}


df_en['lang'] = df_en.lang.replace(label_dict)
df_de['lang'] = df_de.lang.replace(label_dict)
df_fr['lang'] = df_fr.lang.replace(label_dict)

In [11]:
# languages
# q3 ['en', 'de', 'es', 'it', 'fr', 'nl']
# q4 ['en', 'de', 'es', 'it', 'fr', 'nl', 'ro']
# q5 ['en', 'de', 'es', 'it', 'fr', 'nl', 'ro']
# q6 ['en', 'fa', 'de', 'es', 'it', 'fr', 'nl', 'ro']
# q7 ['en', 'de', 'it', 'fr']
# q8 ['en']

In [12]:
# EN
# q3 0.595
# q4 0.72
# q5 0.593
# q6 0.59
# q7 0.558
# q8 0.46

In [13]:
# FR
# q3 0.545
# q4 0.60
# q5 0.566
# q6 0.52
# q7 0.65
# q8 

In [14]:
# DE
# q3 0.656
# q4 0.78
# q5 0.673
# q6 0.54
# q7 0.62
# q8 

In [15]:
accuracy_score(df_en["lang"], re_en)

0.4634146341463415

In [16]:
accuracy_score(df_fr["lang"], re_fr)

/home/oshara/Programs/anaconda2/envs/jupy/lib/python3.7/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/oshara/Programs/anaconda2/envs/jupy/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan

In [17]:
accuracy_score(df_de["lang"], re_de)

nan